In [35]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [36]:
#import iris data
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
  
# metadata 
print(iris.metadata) 
  
# variable information 
print(iris.variables) 

{'uci_id': 53, 'name': 'Iris', 'repository_url': 'https://archive.ics.uci.edu/dataset/53/iris', 'data_url': 'https://archive.ics.uci.edu/static/public/53/data.csv', 'abstract': 'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 150, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1936, 'last_updated': 'Tue Sep 12 2023', 'dataset_doi': '10.24432/C56C76', 'creators': ['R. A. Fisher'], 'intro_paper': {'title': 'The Iris data set: In search of the source of virginica', 'authors': 'A. Unwin, K. Kleinman', 'published_in': 'Significance, 2021', 'year': 2021, 'url': 'https://www.semanticscholar.org/paper/4599862ea877863669a6a8e63a3c707a787d5d7e', 'doi': '1740-9713.01589'}, 'add

In [37]:
#split dataset into shadow and target datasets

from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# "shadow" dataset
shadow_X, shadow_y = X_train, y_train

# "target" dataset
target_X, target_y = X_test, y_test

In [38]:
#create original target model
X_train, X_test, y_train, y_test = train_test_split(target_X, target_y, test_size=0.2, random_state=42)

target_original_model = LogisticRegression()
target_original_model.fit(X_train, y_train)

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [39]:
#create unlearned target model
index_unlearned = np.random.randint(0, len(target_X))

target_unlearned_X = target_X.drop(index=index_unlearned)
target_unlearned_y = target_y.drop(index=index_unlearned)

X_train_unlearned, X_test_unlearned, y_train_unlearned, y_test_unlearned = train_test_split(target_unlearned_X, target_unlearned_y, test_size=0.2, random_state=42)

target_unlearned_model = LogisticRegression()
target_unlearned_model.fit(X_train_unlearned, y_train_unlearned)

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [74]:
#split shadow dataset into positive and negative

#here,the code does a 80-20 split (positive recieves 80% and negative recieves 20%)
shadow_X_train, shadow_X_test, shadow_y_train, shadow_y_test = train_test_split(shadow_X, shadow_y, test_size=0.2, random_state=42)

#shadow "positive"
positive_shadow_X, positive_shadow_y = shadow_X_train, shadow_y_train

#shadow "negative"
negative_shadow_X, negative_shadow_y = shadow_X_test, shadow_y_test

In [75]:
#create original shadow model

X_train, X_test, y_train, y_test = train_test_split(positive_shadow_X, positive_shadow_y, test_size=0.2, random_state=42)

shadow_original_model = LogisticRegression()
shadow_original_model.fit(X_train, y_train)

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [76]:
#create unlearned shadow models

shadow_unlearned_models = {}

for i in range((len(positive_shadow_X) // 4)):
    temp_X = np.delete(positive_shadow_X, i, axis=0)
    temp_y = np.delete(positive_shadow_y, i, axis=0)
    
    model = LogisticRegression()
    
    model.fit(temp_X, temp_y)
    
    shadow_unlearned_models[f"shadow_unlearned_model_index_{i}"] = model

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

In [89]:
#generate positive posteriors 
positive_posteriors_list = [[], []]  

for i in range((len(positive_shadow_X) // 4)):
    unlearned_model = shadow_unlearned_models[f"shadow_unlearned_model_index_{i}"]

    original_model_prob = shadow_original_model.predict_proba([positive_shadow_X.iloc[i]])[0]
    unlearned_model_prob = unlearned_model.predict_proba([positive_shadow_X.iloc[i]])[0]
    
    positive_posteriors_list[0].append(original_model_prob)
    positive_posteriors_list[1].append(unlearned_model_prob)

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.

In [90]:
#generate negative posteriors 
negative_posteriors_list = [[], []]  

for i in range(len(negative_shadow_X)):
    unlearned_model = shadow_unlearned_models[f"shadow_unlearned_model_index_{i}"]

    original_model_prob = shadow_original_model.predict_proba([negative_shadow_X.iloc[i]])[0]
    unlearned_model_prob = unlearned_model.predict_proba([negative_shadow_X.iloc[i]])[0]
    
    negative_posteriors_list[0].append(original_model_prob)
    negative_posteriors_list[1].append(unlearned_model_prob)

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.

In [94]:
#use Direct concatenate to make the feature (using Sorted difference would most likely work the best for Logistic Regression however - sec 5.3)

posteriors_original = []
posteriors_original.extend(positive_posteriors_list[0])
posteriors_original.extend(negative_posteriors_list[0])

posteriors_unlearned = []
posteriors_unlearned.extend(positive_posteriors_list[0])
posteriors_unlearned.extend(negative_posteriors_list[1])

posteriors_unlearned

[array([9.66055700e-01, 3.39415834e-02, 2.71669597e-06]),
 array([0.00077611, 0.25294818, 0.74627572]),
 array([9.53844382e-01, 4.61473179e-02, 8.29960536e-06]),
 array([1.74460604e-04, 1.33840572e-01, 8.65984967e-01]),
 array([0.02067828, 0.78897662, 0.1903451 ]),
 array([9.51733689e-01, 4.82414519e-02, 2.48590865e-05]),
 array([3.43238628e-04, 1.08704582e-01, 8.90952179e-01]),
 array([9.34941249e-05, 4.73833363e-02, 9.52523170e-01]),
 array([0.06035366, 0.87061291, 0.06903344]),
 array([0.01293498, 0.6904367 , 0.29662832]),
 array([9.56476344e-01, 4.35116406e-02, 1.20149093e-05]),
 array([9.56589857e-01, 4.34024723e-02, 7.67090425e-06]),
 array([1.21166286e-04, 5.35568470e-02, 9.46321987e-01]),
 array([0.00975215, 0.6978484 , 0.29239945]),
 array([2.56980021e-04, 1.04898131e-01, 8.94844889e-01]),
 array([0.1493204 , 0.83676523, 0.01391437]),
 array([0.03091224, 0.86154219, 0.10754557]),
 array([9.26835421e-01, 7.31597956e-02, 4.78379371e-06]),
 array([0.01683426, 0.65737449, 0.325791